In [1]:
'''
1. Load 2 template models T3 and T3 and 
   Load PhGecPh ["Phenomenal" to "Generative with Context" to "Phenomenal" spaces] Model 
2. Calculate phenomenal (ph) T2, T3 values from template models
3. Graph template data
5. Graph T2, T3 context curves for different time step outputs, from the geContext Model
6. Calculate exploratory context curves for a timestep (timeIndices Index)
    using a set of weights to offset from the context average toward the T2 and T3 context boundaries
7. Graph exploritory Ge Context values for a time step 
8. Calculate Ph values for exploratory time and context values  
9. Graph exploritory Ph curve against T2 and T3 reference shapes for a time and context vector
10. Generate blender json data to graph phenomenal (leaf shapes) context exploration in 3D

'''
pass

In [2]:
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import numpy as np
import json 
import math as m
import random
import time
import NNArt as gy
from sympy.utilities.iterables import multiset_permutations

import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio

CURRENT_MODEL_SAVE_DIR = './save/'

offline.init_notebook_mode(connected=True) 
np.set_printoptions(precision=3, suppress=True)

In [3]:
class Net(nn.Module):
    def __init__(self, layerSizes, name):
        super(Net, self).__init__()
        self.shape = layerSizes
        self.name = name
        self.seq = nn.Sequential(
            nn.Linear(layerSizes[0],layerSizes[1]),
            nn.ELU(),          
            nn.Linear(layerSizes[1],layerSizes[2]),
            nn.ELU(),            
            nn.Linear(layerSizes[2],layerSizes[3]),
            nn.ELU()
        )          
    def forward(self, x):
        return self.seq(x) 

In [4]:
def createContextValues(timeIndices=[0.,.5,1.], c1Range=[.2,.1], c2Range=[.1,0.], numLines=3):
    '''
    timeIndices[0] needs to be 0 (need to fix)
    returns (numLines, numTimeIndices, 1 )
    
    '''    
    clist=[]
    for y1 in np.linspace(c1Range[0],c1Range[1],numLines):
        for y2 in np.linspace(c2Range[0],c2Range[1],numLines):
            m = (y1-y2)/(timeIndices[0]-timeIndices[-1]) #(y-y)/(x-x) slope 
            #if timeIndices[0]==0, y1 is yintercept
            c = np.array([[m*x+y1] for x in timeIndices]) #c values for time indices
            clist.append(c)
            
    return np.array(clist) 


In [5]:
def getContextSearchVectors(phTemplateModels, geContextModel, geTensor, timeIndex):
    '''
    generate a context avg and search vectors for each template context
        the search vector is the delta between the template context and the average of the templates
        Create Ph data from templates and geTensor to generating geCn values 
        Use geCn values to generate average and delta values for timeIndex

    '''        

    #translate geTensor to template Ph values
    tphT0Out = phTemplateModels[0](torch.Tensor(geTensor))
    tphT1Out = phTemplateModels[1](torch.Tensor(geTensor))
    ph0TOut = tphT0Out.data.numpy()
    ph1TOut = tphT1Out.data.numpy()

    #geContextModel output from template generated ph values
    tgeCn0 = geContextModel(tphT0Out)
    tgeCn1 = geContextModel(tphT1Out)
    geCn0 = tgeCn0.data.numpy()
    geCn1 = tgeCn1.data.numpy()

    gcAvg = ((geCn0+geCn1)/2)[timeIndex,:,2] #average of T0 and T1 values
    gcDelta0 = (geCn0[timeIndex,:,2] - gcAvg) #delta between T0 and average
    gcDelta1 = (geCn1[timeIndex,:,2] - gcAvg) #delta between T1 and average
    
    return gcAvg, np.array([gcDelta0, gcDelta1])


In [6]:
def createPhFromContext(phContextModel, contextCurves, geTensor, timeIndex):
    '''
    Take geTensor and add context values for each context array
    Then use the phContext model to translate it into Ph space
    
    contextCurves is a list of context values that match the numMorphSteps (i.e. geTensor.shape[1])
    
    '''
    if geTensor.shape[1] != len(contextCurves[0]):
        print('The morph dimension of geMatrix: {}, must equal the number of context values {}'
              .format(geTensor.shape[1], len(contextCurves[0]))) 
        
    geList=[]
    for c in contextCurves:
        cValue = np.reshape(c,[9,-1])
        #add context to ge matrix
        geList.append(np.concatenate((geTensor[timeIndex],cValue), axis=1))
    geExplore = np.array(geList)   

    #send through ph model to get ph values based on ge with context
    return phContextModel(torch.Tensor(geExplore)).data.numpy()


In [7]:
def createPhFromContextAcrossTime(phTemplateModels, geContextModel, phContextModel, geTensor, contextWeights):
    '''
    phTemplateModels [ph2tModel,ph3tModel] 
    geContextModel.shape [21, 15, 10, 3]
    phContextModel.shape [3, 10, 15, 21]
    geTensor [timeIndex, morphIndex, tm values (2)] (9, 9, 2)
    return phTensors [timeIndex, morphIndex, 3d morph values]
    
    '''
    pList=[]
    for t in range(geTensor.shape[0]):
        gcAvg, [gcDelta0, gcDelta1] = getContextSearchVectors(phTemplateModels, geCnModel, geTensor, t)
        context = [contextWeights[0]*gcDelta0+contextWeights[1]*gcDelta1+gcAvg]
        phExplore = createPhFromContext(phContextModel, context, geTensor, t)[:,1:] #suppress first value (just offset)
        pList.append(phExplore[0])
    return np.array(pList)
    
# phEx = createPhFromContextAcrossTime([ph2tModel,ph3tModel],geCnModel, phCnModel, geIn, [1.3,0.])
# geCnModel.shape,phCnModel.shape,geIn.shape, phEx.shape

In [8]:
'''
1. Load 2 template models and PhGecPh Models

'''

ph0tModel = Net([2,10,15,21],'ph0tModel')
ph1tModel = Net([2,10,15,21],'ph1tModel')

path=CURRENT_MODEL_SAVE_DIR+'templateModel'
ph0tModel.load_state_dict(torch.load(path+'0.pth'))
ph1tModel.load_state_dict(torch.load(path+'1.pth'))

PATH = CURRENT_MODEL_SAVE_DIR +'REL PhRcPh-T0T1-mask 11p05.pth'
relCnModel = Net([21,15,10,3],'relCnModel')
relCnModel.load_state_dict(torch.load(PATH))

PATH = CURRENT_MODEL_SAVE_DIR +'PH PhRcPh-T0T1-mask 11p05.pth'
phCnModel = Net([3,10,15,21],'phCnModel')
phCnModel.load_state_dict(torch.load(PATH))

ph0tModel.eval()
ph1tModel.eval()
relCnModel.eval()

ph0tModel.shape, ph1tModel.shape,

([2, 10, 15, 21], [2, 10, 15, 21])

In [9]:
'''
2. Calculate ph T2, T3 values from templates

'''

morphIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
timeIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
relIn = gy.createRmInput(morphIndices, timeIndices, sPermutation=[])
relIn01 = gy.createRmInput(morphIndices, timeIndices, sPermutation=[0.,1.])
relIn10 = gy.createRmInput(morphIndices, timeIndices, sPermutation=[1.,0.])

ph0t = ph0tModel(torch.Tensor(relIn)).data.numpy()
ph1t = ph1tModel(torch.Tensor(relIn)).data.numpy()

#geC values from template inputs
trelCn0 = relCnModel(torch.Tensor(ph0t))
trelCn1 = relCnModel(torch.Tensor(ph1t))
relCn0 = trelCn0.data.numpy()
relCn1 = trelCn1.data.numpy()

relIn.shape,relIn01.shape, ph0t.shape

((9, 9, 2), (9, 9, 4), (9, 9, 21))

In [10]:
'''
3. Graph template data

'''

grPh0t = np.reshape(ph0t,(-1,21))
grPh1t = np.reshape(ph1t,(-1,21))
fdata = gy.FormatNetworkToGraphColumns(grPh1t, xOffset=0, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.5,1.1],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.8,1.2],
    )
)
f = gy.graphCurves(fdata, layout, dashRange=range(0,9))

offline.iplot(f)

In [11]:


labels=['Temp0 T=.1', 'Temp1 T=.1',
        'Temp0 T=.3', 'Temp1 T=.3',
        'Temp0 T=.5', 'Temp1 T=.5', 
        'Temp0 T=.7', 'Temp1 T=.7',
        'Temp0 T=.9', 'Temp1 T=.9',
        'Temp0 T=1.0', 'Temp1 T=1.0',
        'Temp0 T=1.1', 'Temp1 T=1.1']
layout?

In [12]:
'''
5. Graph T0, T1 context curves for different time step outputs, from the relContext Model

'''
gx0y1 = np.array([relIn[0,1:,1],relCn0[1,1:,2]])
gx1y1 = np.array([relIn[0,1:,1],relCn1[1,1:,2]])

gx0y3 = np.array([relIn[0,1:,1],relCn0[3,1:,2]])
gx1y3 = np.array([relIn[0,1:,1],relCn1[3,1:,2]])

gx0y4 = np.array([relIn[0,1:,1],relCn0[4,1:,2]])
gx1y4 = np.array([relIn[0,1:,1],relCn1[4,1:,2]])

gx0y5 = np.array([relIn[0,1:,1],relCn0[5,1:,2]])
gx1y5 = np.array([relIn[0,1:,1],relCn1[5,1:,2]])

gx0y6 = np.array([relIn[0,1:,1],relCn0[6,1:,2]])
gx1y6 = np.array([relIn[0,1:,1],relCn1[6,1:,2]])

gx0y7 = np.array([relIn[0,1:,1],relCn0[7,1:,2]])
gx1y7 = np.array([relIn[0,1:,1],relCn1[7,1:,2]])

gx0y8 = np.array([relIn[0,1:,1],relCn0[8,1:,2]])
gx1y8 = np.array([relIn[0,1:,1],relCn1[8,1:,2]])

glist = [gx0y1,gx1y1, gx0y3,gx1y3, gx0y4,gx1y4, gx0y5,gx1y5 , gx0y6,gx1y6,]

labels=['Temp0 T=.1', 'Temp1 T=.1',
        'Temp0 T=.3', 'Temp1 T=.3',
        'Temp0 T=.5', 'Temp1 T=.5', 
        'Temp0 T=.7', 'Temp1 T=.7',
        'Temp0 T=.9', 'Temp1 T=.9',
        'Temp0 T=1.0', 'Temp1 T=1.0',
        'Temp0 T=1.1', 'Temp1 T=1.1']

layout = go.Layout(
    title = '',
    height = 500,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Morphology index',
        range = [-.01,1.11],
        tick0 = 0.,
        dtick = 0.2
    ),
    yaxis=dict(
        title= 'Context',
        range = [-.4,.5],
    )    
)
layout.title = 'Figure 4. Context changes over morphology steps for <br>different template and time indexes' 
f = gy.graphCurves(glist, layout, labels, 
        dashRange=gy.rangeList([range(2,4),range(6,8),range(10,112)]))

offline.iplot(f)
# if not os.path.exists('images'):
#     os.mkdir('images')
# pio.write_image(f, './images/fig4.png')

In [23]:
'''
6. Calculate exploratory context curves for a timestep (timeIndices Index)
    using a set of weights to offset from the context average toward the T0 and T1 context boundaries

'''
timeIndex = 5
rcAvg, [rcDelta2, rcDelta3] = getContextSearchVectors([ph0tModel,ph1tModel], relCnModel, relIn, timeIndex)

# weights0 = [1., .5, 1.5, 2.5, 3.5]
# weights1 = [1., 0., .5, 1.5, 2.5, 3.5]
weights0 = [0]
weights1 = [.5,1., 1.5, 2., 2.5, .3]
context0 = [w*rcDelta2+rcAvg for w in  weights0]
context1 = [w*rcDelta3+rcAvg for w in  weights1]

np.array(context0).shape

(1, 9)

In [14]:
# str(timeIndices[timeIndex])

In [15]:
'''
7. Graph exploritory Relational Context values for a time step 

'''

rcxy0 = np.array([[relIn[0,1:,1], c]for c in np.array(context0)[:,1:]])
rcxy1 = np.array([[relIn[0,1:,1], c]for c in np.array(context1)[:,1:]])

rlist = np.concatenate((rcxy0,rcxy1))

layout = go.Layout(
    title = '',
    height = 500,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Morphology index',
        range = [-.01,1.2],
        tick0 = 0.,
        dtick = 0.2
    ),
    yaxis=dict(
        title= 'Context',
        range = [-.4,.2],
    )    
)
layout.title = 'Figure 5. Context interpolation/extrapolation<br> of morphology for time index: ' + str(timeIndices[timeIndex])
f = gy.graphCurves(rlist, layout, dashRange=[0,5],
                   labels=['Template0', 'Temp0 .5', 'Temp0 1.5','Temp0 2.5', 'Temp0 3.5', 'Template1','Avg',
                           'Temp1 .5', 'Temp1 1.5', 'Temp1 2.5','Temp1 3.5', ],
                   )

offline.iplot(f)
if not os.path.exists('images'):
    os.mkdir('images')
pio.write_image(f, './images/fig5.png')

In [20]:
'''
8. Calculate Ph values for exploratory time and context values  

'''
timeIndex=6
c0 = np.array(context0)
c1 =  np.array(context1)
contextCurves = np.concatenate((c0,c1))

#ph from raw template
pht0Petal = ph0t[timeIndex,1:]#suppress first morph value (just offset)
pht1Petal = ph1t[timeIndex,1:]

#ph from context model and exxloritory context curves
phExplore = createPhFromContext(phCnModel,contextCurves,relIn,timeIndex)[:,1:] #suppress first value (just offset)

c0.shape, c1.shape, contextCurves.shape, pht0Petal.shape, phExplore.shape

((1, 9), (6, 9), (7, 9), (8, 21), (7, 8, 21))

In [25]:
'''
9. Graph exploritory Ph curve against T0 and T1 reference shapes for a time and context vector

'''
###############
cIndex = 0 #Context index selecting one of the exploritory context vectors
###############

nx = np.concatenate((pht0Petal, pht1Petal,phExplore[cIndex], ))
gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.1,1.5],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.5,1.],
    )
)

layout.title = 'Compare template and exploritory context models xz front view'
f = gy.graphCurves(gx, layout, dashRange=range(0,18))
offline.iplot(f)

In [27]:
nx1 = np.concatenate((pht0Petal, pht1Petal, ))
gx = gy.FormatNetworkToGraphColumns(nx1, xOffset=0, yOffset=7, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 800,
    width = 800,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.0,1.2],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.8,.8],
    )
)
layout.title = 'Compare template models xy top view'
f = gy.graphCurves(gx, layout, dashRange=range(0,9))
offline.iplot(f)

# if not os.path.exists('images'):
#     os.mkdir('images')
# pio.write_image(f, './images/fig2a.png')

In [28]:

gx = gy.FormatNetworkToGraphColumns(nx, xOffset=7, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.5,.5],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.2,.5],
    )
)

layout.title = 'Compare template and exploritory context models yz side view'
f = gy.graphCurves(gx, layout, dashRange=range(0,18))
offline.iplot(f)

In [72]:
timeIndex = 5
rcAvg, [rcDelta2, rcDelta3] = getContextSearchVectors([ph0tModel,ph1tModel], relCnModel, relIn, timeIndex)

# weights0 = [1., .5, 1.5, 2.5, 3.5]
# weights1 = [1., 0., .5, 1.5, 2.5, 3.5]
# weights0 = [0]
# weights1 = [.5,1., 1.5, 2., 2.5, .3]
weights0 = [0]
weights1 = [1., 1.4, 1.7, 1.8, 2.9, 3.4]

context0 = [w*rcDelta2+rcAvg for w in  weights0]
context1 = [w*rcDelta3+rcAvg for w in  weights1]

np.array(context0).shape, np.array(context1).shape

((1, 9), (6, 9))

In [73]:
c0 = np.array(context0)
c1 = np.array(context1)
contextCurves = np.concatenate((c0,c1))

#ph from raw template
pht0Petal = ph0t[timeIndex,1:]#suppress first morph value (just offset)
pht1Petal = ph1t[timeIndex,1:]

#ph from context model and exxloritory context curves
phExplore = createPhFromContext(phCnModel,contextCurves,relIn,timeIndex)[:,1:] #suppress first value (just offset)

timeIndices[timeIndex],contextCurves.shape, phExplore.shape

(0.7, (7, 9), (7, 8, 21))

In [74]:
'''
10. Generate blender json data to graph phenomenal (leaf shapes) context exploration in 3D

x.shape [num petals (time steps), num morph steps, 21 coord]

'''

x = phExplore
# x = createPhFromContextAcrossTime([ph2tModel,ph3tModel],geCnModel, phCnModel, geIn, [1.3,0.])

fb = gy.FormatNetworkToBlender(x, xOffset=0, yOffset=7, zOffset=14, colPoints=7)

PATH = CURRENT_MODEL_SAVE_DIR +'btest.json'
with open(PATH, 'w') as fp:
    json.dump(fb.tolist(),fp)
    
x.shape, PATH

((7, 8, 21), './save/btest.json')